In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import pickle
import time
import math
from math import sin
from math import cos
from math import pi
from minimax_tilting_sampler import TruncatedMVN # for truncated MVNs
import statsmodels.api as sm
from scipy import stats


In [103]:
############ main study
def ge_right(n,age_min,age_max,mean_x,sigma_x,sigma_ep):

    ## personal measures
    mean_x=np.array(mean_x)
    lb = np.zeros_like(mean_x)
    ub = np.ones_like(mean_x) * np.inf
    tmvn = TruncatedMVN(mean_x, sigma_x, lb, ub)
    samples = tmvn.sample(n)
    per_x=samples.T   

    ## error terms
    mean_ep=np.zeros_like(mean_x)

    epsilon_x=np.random.multivariate_normal(mean_ep, sigma_ep, (n,), 'raise') 

    x1=np.array(per_x)
    x2=np.array(epsilon_x)

    
    age=np.random.uniform(age_min,age_max,n)
    race=np.random.binomial(1, 0.16, n)
    x1[:,1]=x1[:,1]+0.1*(race-0.16)

    ## nearest monitor measures
    x_nm=np.zeros_like(x1)
    for i in range(n):
        trig=0
        while trig==0:
            x_nm[i,:]=x1[i,:]+x2[i,:]

            if (x_nm[i,:].min()<0):
                epsilon_x=np.random.multivariate_normal(mean_ep, sigma_ep, (1,), 'raise')
                x2[i,:]=epsilon_x
            else:
                trig=1

    data3=np.concatenate([x1,x_nm,age.reshape(-1,1),race.reshape(-1,1)],axis=1)
    return data3

def ge_main(n,age_min,age_max,mean_x,b_0,b_x,b_w,sd_xi,sigma_x,sigma_ep,linearg=True):

    ## race age error in main model
    ep_xixi=np.random.normal(0,sd_xi,n)
    ## personal, nearest monitor
    data2=ge_right(n,age_min,age_max,mean_x,sigma_x,sigma_ep)
    ######################### 'g()' function
    w=data2[:,np.concatenate([range(n_ele),[2*n_ele],[2*n_ele+1]],axis=0)]
    w = np.delete(w, 1, axis=1)
    if linearg==True:
        g=b_0+w@np.array(b_w)
    else:
        g=b_0+b_w[0]*np.exp(500*(w[:,0]-0.02))/(1+np.exp(500*(w[:,0]-0.02)))+b_w[4]*np.exp(20*(w[:,4]-0.4))/(1+np.exp(20*(w[:,4]-0.4)))+b_w[7]*np.exp(30*(w[:,7]-0.3))/(1+np.exp(30*(w[:,7]-0.3)))+b_w[-1]*(w[:,-1])

    ### generate Y
    Y=b_x*data2[:,1]+g+ep_xixi
    ### partial R2 of X
    R2_x=((b_x*data2[:,1]).var())/(Y.var())
    ### R2
    R2_f=((b_x*data2[:,1]+g).var())/(Y.var())
    ### partial R2 of 'g()' function
    R2_g=((g).var())/(Y.var())

    final=[Y,data2,R2_x,R2_f,R2_g]
    return final

In [107]:
mean_x=np.array([0.00968,0.737,0.019,0.00145,0.011,0.128,0.00130,-0.00129,0.0922,0.0656,0.00118,0.14])
b_x=8
b_w=np.array([0,0,0,0,22,0,0,0,0,0,0,0,0])
sigma_x=np.loadtxt("per_sigma.txt")
sigma_ep=np.loadtxt("error_sigma_rho0.4.txt")
age_min=18
age_max=91

n1=350
n2=1000

b_0=1
sd_xi=9 # sd for the error in main model

In [114]:
v=ge_right(n1,age_min,age_max,mean_x,sigma_x,sigma_ep)
m=ge_main(n2,age_min,age_max,mean_x,b_0,b_x,b_w,sd_xi,sigma_x,sigma_ep,True)

In [123]:
w1_nm_v=v[:,(mean_x.shape[0]):(2*mean_x.shape[0])]
w1_nm_v[[1,2],:]

array([[0.0322944 , 1.10090027, 0.64480114, 0.07906713, 0.94921315,
        0.05689159, 0.03315224, 0.0241754 , 0.12471087, 0.07689717,
        0.00641105, 0.2373747 ],
       [0.01760744, 1.63454941, 0.67010301, 0.06823772, 0.32003816,
        0.14140218, 0.06869768, 0.07280964, 0.12508717, 0.0240432 ,
        0.00780792, 0.2561562 ]])

In [127]:
v=ge_right(n1,age_min,age_max,mean_x,sigma_x,sigma_ep)

x_per_v=v[:,1]
x_nm_v=v[:,1+mean_x.shape[0]]
w1_per_v=v[:,0:mean_x.shape[0]]
w1_per_v=np.delete(w1_per_v, 1, axis=1)
w1_nm_v=v[:,(mean_x.shape[0]):(2*mean_x.shape[0])]
w1_nm_v=np.delete(w1_nm_v, 1, axis=1)
w2_v=v[:,[2*mean_x.shape[0],2*mean_x.shape[0]+1]]


In [130]:
y_m=m[0]
y_m

array([ 1.46786554e+01,  2.23984762e+01,  8.73341872e+00, -1.87047447e+00,
        4.94182355e+00,  1.37998835e+01,  8.55525077e+00,  2.13669396e+01,
       -4.78925838e-01,  7.08611007e+00, -2.03975169e+00,  3.80374046e+00,
        1.76219464e+01,  2.52515747e+01,  1.75090927e+01, -1.83337093e-01,
       -6.92909918e+00,  6.01642665e+00, -3.27972288e+00,  7.00317442e+00,
        2.30429314e+01, -2.72563733e+00,  1.68198063e+01,  1.51322074e+01,
        1.20616979e+01, -8.94810564e-01,  1.01866660e+01,  8.22574025e+00,
        2.73714504e+01,  2.66720654e+01, -9.77046695e-01,  1.34732774e+01,
       -4.54936013e+00,  3.29697477e+01,  3.45110973e+00,  1.13686704e+01,
        1.56485178e+01,  1.67552574e+01,  1.90724754e+01,  1.75981000e+01,
        1.40838355e+01,  2.95118616e+00,  1.85945796e+01,  5.78960162e+00,
        1.86540760e+00,  1.04057096e+01,  2.03799120e+01, -5.66881236e+00,
        2.23950474e+01, -1.69709036e+00,  1.17487625e+01,  7.47765702e+00,
        7.01437352e+00,  

In [4]:

######## main study dataset generation#####################

### generate personal measures, error terms, and nearest monitor measures in ME models
def ge_right(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,

             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2):

    ## generate personal measures
    mean_x_ca7=np.array([mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca])
    sigma_x_ca7=np.array([[sigma_x**2,cor12*sigma_x*sigma_cu,cor13*sigma_x*sigma_s,
                          cor14*sigma_x*sigma_zn,cor15*sigma_x*sigma_ni,cor16*sigma_x*sigma_al,
                          cor17*sigma_x*sigma_ca],

                          [cor12*sigma_x*sigma_cu,sigma_cu**2,cor23*sigma_cu*sigma_s,
                          cor24*sigma_cu*sigma_zn,cor25*sigma_cu*sigma_ni,cor26*sigma_cu*sigma_al,
                          cor27*sigma_cu*sigma_ca],

                          [cor13*sigma_x*sigma_s,cor23*sigma_cu*sigma_s,sigma_s**2,
                          cor34*sigma_s*sigma_zn,cor35*sigma_s*sigma_ni,cor36*sigma_s*sigma_al,
                          cor37*sigma_s*sigma_ca],

                          [cor14*sigma_zn*sigma_x,cor24*sigma_cu*sigma_zn,cor34*sigma_s*sigma_zn,
                          sigma_zn**2,cor45*sigma_zn*sigma_ni,cor46*sigma_zn*sigma_al,
                          cor47*sigma_zn*sigma_ca],

                          [cor15*sigma_ni*sigma_x,cor25*sigma_cu*sigma_ni,cor35*sigma_s*sigma_ni,
                          cor45*sigma_ni*sigma_zn,sigma_ni**2,cor56*sigma_ni*sigma_al,
                          cor57*sigma_ni*sigma_ca],

                          [cor16*sigma_al*sigma_x,cor26*sigma_cu*sigma_al,cor36*sigma_s*sigma_al,
                          cor46*sigma_al*sigma_zn,cor56*sigma_al*sigma_ni,sigma_al**2,
                          cor67*sigma_al*sigma_ca],

                          [cor17*sigma_ca*sigma_x,cor27*sigma_cu*sigma_ca,cor37*sigma_s*sigma_ca,
                          cor47*sigma_ca*sigma_zn,cor57*sigma_ca*sigma_ni,cor67*sigma_al*sigma_ca,
                          sigma_ca**2]])

    # generated by truncated MVNs
    lb = np.zeros_like(mean_x_ca7)
    ub = np.ones_like(mean_x_ca7) * np.inf
    tmvn = TruncatedMVN(mean_x_ca7, sigma_x_ca7, lb, ub)
    samples = tmvn.sample(n)

    per_x_ca7=samples.T   # nx7


    ## generate error terms
    sigma_ep_x=sigma_x*sqrt(1/icc_x-1)
    sigma_ep_cu=sigma_cu*sqrt(1/icc_cu-1)
    sigma_ep_s=sigma_s*sqrt(1/icc_s-1)

    sigma_ep_zn=sigma_zn*sqrt(1/icc_zn-1)
    sigma_ep_ni=sigma_ni*sqrt(1/icc_ni-1)
    sigma_ep_al=sigma_al*sqrt(1/icc_al-1)

    sigma_ep_ca=sigma_ca*sqrt(1/icc_ca-1)

    mean_ep=np.array([mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca])

    sigma_ep=np.array([[sigma_ep_x**2,cor2*sigma_ep_x*sigma_ep_cu,cor2*sigma_ep_x*sigma_ep_s,
                          cor2*sigma_ep_x*sigma_ep_zn,cor2*sigma_ep_x*sigma_ep_ni,cor2*sigma_ep_x*sigma_ep_al,
                          cor2*sigma_ep_x*sigma_ep_ca],

                          [cor2*sigma_ep_x*sigma_ep_cu,sigma_ep_cu**2,cor2*sigma_ep_cu*sigma_ep_s,
                          cor2*sigma_ep_cu*sigma_ep_zn,cor2*sigma_ep_cu*sigma_ep_ni,cor2*sigma_ep_cu*sigma_ep_al,
                          cor2*sigma_ep_cu*sigma_ep_ca],

                          [cor2*sigma_ep_x*sigma_ep_s,cor2*sigma_ep_cu*sigma_ep_s,sigma_ep_s**2,
                          cor2*sigma_ep_s*sigma_ep_zn,cor2*sigma_ep_s*sigma_ep_ni,cor2*sigma_ep_s*sigma_ep_al,
                          cor2*sigma_ep_s*sigma_ep_ca],

                          [cor2*sigma_ep_zn*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_zn,cor2*sigma_ep_s*sigma_ep_zn,
                          sigma_ep_zn**2,cor2*sigma_ep_zn*sigma_ep_ni,cor2*sigma_ep_zn*sigma_ep_al,
                          cor2*sigma_ep_zn*sigma_ep_ca],

                          [cor2*sigma_ep_ni*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_ni,cor2*sigma_ep_s*sigma_ep_ni,
                          cor2*sigma_ep_ni*sigma_ep_zn,sigma_ep_ni**2,cor2*sigma_ep_ni*sigma_ep_al,
                          cor2*sigma_ep_ni*sigma_ep_ca],

                          [cor2*sigma_ep_al*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_al,cor2*sigma_ep_s*sigma_ep_al,
                          cor2*sigma_ep_al*sigma_ep_zn,cor2*sigma_ep_al*sigma_ep_ni,sigma_ep_al**2,
                          cor2*sigma_ep_al*sigma_ep_ca],

                          [cor2*sigma_ep_ca*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_ca,cor2*sigma_ep_s*sigma_ep_ca,
                          cor2*sigma_ep_ca*sigma_ep_zn,cor2*sigma_ep_ca*sigma_ep_ni,cor2*sigma_ep_al*sigma_ep_ca,
                          sigma_ep_ca**2]])

    # generated by multivariate normal (errors)
    epsilon_x_ca7=np.random.multivariate_normal(mean_ep, sigma_ep, (n,), 'raise')   # nx7

    x1=pd.DataFrame(per_x_ca7)
    x2=pd.DataFrame(epsilon_x_ca7)

    x1.columns=['PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per']
    x2.columns=['eps_x','eps_cu','eps_s','eps_zn','eps_ni','eps_al','eps_ca']

    xx=pd.concat([x1,x2],axis=1)

    data2=xx

    # ensure that the nearest monitor measures are all nonnegative
    for i in range(n):
        trig=0
        while trig==0:
            data2.loc[i,'PM2.5_nm']=data2.loc[i,'PM2.5_per']+data2.loc[i,'eps_x']
            data2.loc[i,'Cu_nm']=data2.loc[i,'Cu_per']+data2.loc[i,'eps_cu']
            data2.loc[i,'S_nm']=data2.loc[i,'S_per']+data2.loc[i,'eps_s']
            data2.loc[i,'Zn_nm']=data2.loc[i,'Zn_per']+data2.loc[i,'eps_zn']
            data2.loc[i,'Ni_nm']=data2.loc[i,'Ni_per']+data2.loc[i,'eps_ni']
            data2.loc[i,'Al_nm']=data2.loc[i,'Al_per']+data2.loc[i,'eps_al']
            data2.loc[i,'Ca_nm']=data2.loc[i,'Ca_per']+data2.loc[i,'eps_ca']

            nmi = data2.loc[i,['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]

            if (nmi[nmi<0].count().sum()>0):
                epsilon_x_ca71=np.random.multivariate_normal(mean_ep, sigma_ep, (1,), 'raise')
                #x21=pd.DataFrame(epsilon_x_ca71)
                data2.loc[i,['eps_x','eps_cu','eps_s','eps_zn','eps_ni','eps_al','eps_ca']]=epsilon_x_ca71
            else:
                trig=1
    return data2


## this one may need to be modified
def ge_left(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
            sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

            cor12,cor13,cor14,cor15,cor16,cor17,
            cor23,cor24,cor25,cor26,cor27,
            cor34,cor35,cor36,cor37,
            cor45,cor46,cor47,
            cor56,cor57,
            cor67,

            mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
            icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2):

    data=ge_right(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,

             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2)

    data2=data

    return data2


### generate main study dataset(Y, race, age, g(w1,w2)...) and calculate R_squared
def ge_main(n,age_min,age_max,
            mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
            sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

            cor12,cor13,cor14,cor15,cor16,cor17,
            cor23,cor24,cor25,cor26,cor27,
            cor34,cor35,cor36,cor37,
            cor45,cor46,cor47,
            cor56,cor57,
            cor67,

            mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
            icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2,

            b_cu,b_s,b_zn,b_ni,b_al,b_ca,b_age,b_race,
            b_cu_s,b_zn_ni,b_al_ca,b_s_zn,b_ni_al,b_ca_cu,b_cu2,b_s2,b_zn2,b_ni2,b_al2,b_ca2,
            b_0,b_x,sd_xi,b_ca12,b_s12,b_zn12,b_cu12,b_al12,b_ni12,
            ba_cu,ba_s,ba_zn,ba_ni,ba_al,ba_ca):
    
    ## generate race age and error term in main model
    raced=np.random.binomial(1, 0.16, n)
    aged=np.random.uniform(age_min,age_max,n)
    ep_xixi=np.random.normal(0,sd_xi,n)

    race=pd.DataFrame(raced)
    age=pd.DataFrame(aged)
    error_xi=pd.DataFrame(ep_xixi)

    race.columns=['race']
    age.columns=['age']
    error_xi.columns=['error_xi']

    ## the generated personal, nearest monitor measures 
    ## and error terms in ME models (by ge_left() above)
    data=ge_left(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,

             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2)

    data2=data

    ## put personal, nearest monitor measures, and race, age, error_xi together
    data3=pd.concat([data2,race,age,error_xi],axis=1)

    ######################### calculate g(w1,w2) in main study
    data3['gww']=b_cu*(data3['Cu_per'].apply(lambda X: sin(70*X*pi)))
    +b_s*(data3['S_per'].apply(lambda X: sin(4.2*X*pi)))+b_zn*(data3['Zn_per'].apply(lambda X: sin(9*X*pi)))
    +b_ni*(data3['Ni_per'].apply(lambda X: sin(560*X*pi)))+b_al*(data3['Al_per'].apply(lambda X: sin(30*X*pi)))

    +b_ca*(data3['Ca_per'].apply(lambda X: sin(5.6*X*pi)))+b_race*data3['race']
    +ba_cu*data3['age']*data3['Cu_per']+ba_s*data3['age']*data3['S_per']+ba_zn*data3['age']*data3['Zn_per']
    +ba_ni*data3['age']*data3['Ni_per']+ba_al*data3['age']*data3['Al_per']+ba_ca*data3['age']*data3['Ca_per']
    ############################

    ############### calculate g2 (now in the 3 scenarios the model doesn't have 'g2')
    data3['g2']=b_cu_s*data3['Cu_per']*data3['S_per']+b_zn_ni*data3['Zn_per']*data3['Ni_per']
    +b_al_ca*data3['Al_per']*data3['Ca_per']
    +b_s_zn*data3['S_per']*data3['Zn_per']+b_ni_al*data3['Ni_per']*data3['Al_per']+b_ca_cu*data3['Ca_per']*data3['Cu_per']

    +b_cu2*(data3['Cu_per']**2)+b_s2*(data3['S_per']**2)+b_zn2*(data3['Zn_per']**2)
    +b_ni2*(data3['Ni_per']**2)+b_al2*(data3['Al_per']**2)+b_ca2*(data3['Ca_per']**2)
    ##############

    ## Y
    data3['Y']=b_0+b_x*data3['PM2.5_per']+data3['gww']+data3['error_xi']

    ## R_squared and partial R_squared
    R2_x=((b_x*data3['PM2.5_per']).var())/(data3['Y'].var()) # R2_x

    R2_f=((b_x*data3['PM2.5_per']+data3['gww']).var())/(data3['Y'].var()) # R_squared(total)

    R2_g=((data3['gww']).var())/(data3['Y'].var()) # R2_g

    R2_g2=(data3['g2'].var())/(data3['Y'].var()) # R2_g2 (now the model doesn't have 'g2')

    ## calculate correlations between Y and others in main model
    cor_x_y_p=data3[['Y','PM2.5_per']].corr()
    cor_x_y_s=data3[['Y','PM2.5_per']].corr('spearman')

    cor_cu_y_p=data3[['Y','Cu_per']].corr()
    cor_cu_y_s=data3[['Y','Cu_per']].corr('spearman')

    cor_s_y_p=data3[['Y','S_per']].corr()
    cor_s_y_s=data3[['Y','S_per']].corr('spearman')

    cor_zn_y_p=data3[['Y','Zn_per']].corr()
    cor_zn_y_s=data3[['Y','Zn_per']].corr('spearman')

    cor_ni_y_p=data3[['Y','Ni_per']].corr()
    cor_ni_y_s=data3[['Y','Ni_per']].corr('spearman')

    cor_al_y_p=data3[['Y','Al_per']].corr()
    cor_al_y_s=data3[['Y','Al_per']].corr('spearman')

    cor_ca_y_p=data3[['Y','Ca_per']].corr()
    cor_ca_y_s=data3[['Y','Ca_per']].corr('spearman')

    cor_age_y_p=data3[['Y','age']].corr()
    cor_age_y_s=data3[['Y','age']].corr('spearman')

    cor_race_y_p=data3[['Y','race']].corr()
    cor_race_y_s=data3[['Y','race']].corr('spearman')

    ## put main study dataset, R_squared and partial R_squared, and correlations above in a list and then return
    final=[data3,n,R2_x,R2_f,R2_g,R2_g2,
           cor_x_y_p.iloc[0,1],cor_cu_y_p.iloc[0,1],cor_s_y_p.iloc[0,1],cor_zn_y_p.iloc[0,1],
           cor_ni_y_p.iloc[0,1],cor_al_y_p.iloc[0,1],cor_ca_y_p.iloc[0,1],cor_age_y_p.iloc[0,1],cor_race_y_p.iloc[0,1],
           cor_x_y_s.iloc[0,1],cor_cu_y_s.iloc[0,1],cor_s_y_s.iloc[0,1],cor_zn_y_s.iloc[0,1],
           cor_ni_y_s.iloc[0,1],cor_al_y_s.iloc[0,1],cor_ca_y_s.iloc[0,1],cor_age_y_s.iloc[0,1],cor_race_y_s.iloc[0,1]]

    return final


######## validation study dataset generation#####################

### generate personal measures, error terms, and nearest monitor measures in ME models
def ge_right2(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,

             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2):

    ## generate personal measures
    mean_x_ca7=np.array([mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca])
    sigma_x_ca7=np.array([[sigma_x**2,cor12*sigma_x*sigma_cu,cor13*sigma_x*sigma_s,
                          cor14*sigma_x*sigma_zn,cor15*sigma_x*sigma_ni,cor16*sigma_x*sigma_al,
                          cor17*sigma_x*sigma_ca],

                          [cor12*sigma_x*sigma_cu,sigma_cu**2,cor23*sigma_cu*sigma_s,
                          cor24*sigma_cu*sigma_zn,cor25*sigma_cu*sigma_ni,cor26*sigma_cu*sigma_al,
                          cor27*sigma_cu*sigma_ca],

                          [cor13*sigma_x*sigma_s,cor23*sigma_cu*sigma_s,sigma_s**2,
                          cor34*sigma_s*sigma_zn,cor35*sigma_s*sigma_ni,cor36*sigma_s*sigma_al,
                          cor37*sigma_s*sigma_ca],

                          [cor14*sigma_zn*sigma_x,cor24*sigma_cu*sigma_zn,cor34*sigma_s*sigma_zn,
                          sigma_zn**2,cor45*sigma_zn*sigma_ni,cor46*sigma_zn*sigma_al,
                          cor47*sigma_zn*sigma_ca],

                          [cor15*sigma_ni*sigma_x,cor25*sigma_cu*sigma_ni,cor35*sigma_s*sigma_ni,
                          cor45*sigma_ni*sigma_zn,sigma_ni**2,cor56*sigma_ni*sigma_al,
                          cor57*sigma_ni*sigma_ca],

                          [cor16*sigma_al*sigma_x,cor26*sigma_cu*sigma_al,cor36*sigma_s*sigma_al,
                          cor46*sigma_al*sigma_zn,cor56*sigma_al*sigma_ni,sigma_al**2,
                          cor67*sigma_al*sigma_ca],

                          [cor17*sigma_ca*sigma_x,cor27*sigma_cu*sigma_ca,cor37*sigma_s*sigma_ca,
                          cor47*sigma_ca*sigma_zn,cor57*sigma_ca*sigma_ni,cor67*sigma_al*sigma_ca,
                          sigma_ca**2]])



    # generated by truncated MVNs
    lb = np.zeros_like(mean_x_ca7)
    ub = np.ones_like(mean_x_ca7) * np.inf
    tmvn2 = TruncatedMVN(mean_x_ca7, sigma_x_ca7, lb, ub)
    samples = tmvn2.sample(n)

    per_x_ca7=samples.T   # nx7


    ## generate error terms
    sigma_ep_x=sigma_x*sqrt(1/icc_x-1)
    sigma_ep_cu=sigma_cu*sqrt(1/icc_cu-1)
    sigma_ep_s=sigma_s*sqrt(1/icc_s-1)

    sigma_ep_zn=sigma_zn*sqrt(1/icc_zn-1)
    sigma_ep_ni=sigma_ni*sqrt(1/icc_ni-1)
    sigma_ep_al=sigma_al*sqrt(1/icc_al-1)

    sigma_ep_ca=sigma_ca*sqrt(1/icc_ca-1)

    mean_ep=np.array([mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca])

    sigma_ep=np.array([[sigma_ep_x**2,cor2*sigma_ep_x*sigma_ep_cu,cor2*sigma_ep_x*sigma_ep_s,
                          cor2*sigma_ep_x*sigma_ep_zn,cor2*sigma_ep_x*sigma_ep_ni,cor2*sigma_ep_x*sigma_ep_al,
                          cor2*sigma_ep_x*sigma_ep_ca],

                          [cor2*sigma_ep_x*sigma_ep_cu,sigma_ep_cu**2,cor2*sigma_ep_cu*sigma_ep_s,
                          cor2*sigma_ep_cu*sigma_ep_zn,cor2*sigma_ep_cu*sigma_ep_ni,cor2*sigma_ep_cu*sigma_ep_al,
                          cor2*sigma_ep_cu*sigma_ep_ca],

                          [cor2*sigma_ep_x*sigma_ep_s,cor2*sigma_ep_cu*sigma_ep_s,sigma_ep_s**2,
                          cor2*sigma_ep_s*sigma_ep_zn,cor2*sigma_ep_s*sigma_ep_ni,cor2*sigma_ep_s*sigma_ep_al,
                          cor2*sigma_ep_s*sigma_ep_ca],

                          [cor2*sigma_ep_zn*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_zn,cor2*sigma_ep_s*sigma_ep_zn,
                          sigma_ep_zn**2,cor2*sigma_ep_zn*sigma_ep_ni,cor2*sigma_ep_zn*sigma_ep_al,
                          cor2*sigma_ep_zn*sigma_ep_ca],

                          [cor2*sigma_ep_ni*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_ni,cor2*sigma_ep_s*sigma_ep_ni,
                          cor2*sigma_ep_ni*sigma_ep_zn,sigma_ep_ni**2,cor2*sigma_ep_ni*sigma_ep_al,
                          cor2*sigma_ep_ni*sigma_ep_ca],

                          [cor2*sigma_ep_al*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_al,cor2*sigma_ep_s*sigma_ep_al,
                          cor2*sigma_ep_al*sigma_ep_zn,cor2*sigma_ep_al*sigma_ep_ni,sigma_ep_al**2,
                          cor2*sigma_ep_al*sigma_ep_ca],

                          [cor2*sigma_ep_ca*sigma_ep_x,cor2*sigma_ep_cu*sigma_ep_ca,cor2*sigma_ep_s*sigma_ep_ca,
                          cor2*sigma_ep_ca*sigma_ep_zn,cor2*sigma_ep_ca*sigma_ep_ni,cor2*sigma_ep_al*sigma_ep_ca,
                          sigma_ep_ca**2]])

    # generated by multivariate normal (errors)
    epsilon_x_ca7=np.random.multivariate_normal(mean_ep, sigma_ep, (n,), 'raise')   # nx7

    x1=pd.DataFrame(per_x_ca7)
    x2=pd.DataFrame(epsilon_x_ca7)

    x1.columns=['PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per']
    x2.columns=['eps_x','eps_cu','eps_s','eps_zn','eps_ni','eps_al','eps_ca']

    xx=pd.concat([x1,x2],axis=1)

    data2=xx

    # ensure that the nearest monitor measures are all nonnegative
    for i in range(n):
        trig=0
        while trig==0:
            data2.loc[i,'PM2.5_nm']=data2.loc[i,'PM2.5_per']+data2.loc[i,'eps_x']
            data2.loc[i,'Cu_nm']=data2.loc[i,'Cu_per']+data2.loc[i,'eps_cu']
            data2.loc[i,'S_nm']=data2.loc[i,'S_per']+data2.loc[i,'eps_s']
            data2.loc[i,'Zn_nm']=data2.loc[i,'Zn_per']+data2.loc[i,'eps_zn']
            data2.loc[i,'Ni_nm']=data2.loc[i,'Ni_per']+data2.loc[i,'eps_ni']
            data2.loc[i,'Al_nm']=data2.loc[i,'Al_per']+data2.loc[i,'eps_al']
            data2.loc[i,'Ca_nm']=data2.loc[i,'Ca_per']+data2.loc[i,'eps_ca']
            nmi = data2.loc[i,['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]

            if (nmi[nmi<0].count().sum()>0):
                epsilon_x_ca71=np.random.multivariate_normal(mean_ep, sigma_ep, (1,), 'raise')
                data2.loc[i,['eps_x','eps_cu','eps_s','eps_zn','eps_ni','eps_al','eps_ca']]=epsilon_x_ca71
            else:
                trig=1
    return data2


## this one may need to be modified
def ge_left2(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,

             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2):

    ## the generated personal measures, nm measures, and error terms in ME models (by ge_right2() above)
    data=ge_right(n,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,

             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,

             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2)

    data2=data

    ## calculate the correlations between PM2.5 and others in personal and nearest monitor levels
    cor_x_cu_p=data2[['Cu_per','PM2.5_per']].corr()
    cor_x_s_p=data2[['S_per','PM2.5_per']].corr()
    cor_x_zn_p=data2[['Zn_per','PM2.5_per']].corr()
    cor_x_ni_p=data2[['Ni_per','PM2.5_per']].corr()
    cor_x_al_p=data2[['Al_per','PM2.5_per']].corr()
    cor_x_ca_p=data2[['Ca_per','PM2.5_per']].corr()

    cor_x_cu_s=data2[['Cu_per','PM2.5_per']].corr('spearman')
    cor_x_s_s=data2[['S_per','PM2.5_per']].corr('spearman')
    cor_x_zn_s=data2[['Zn_per','PM2.5_per']].corr('spearman')
    cor_x_ni_s=data2[['Ni_per','PM2.5_per']].corr('spearman')
    cor_x_al_s=data2[['Al_per','PM2.5_per']].corr('spearman')
    cor_x_ca_s=data2[['Ca_per','PM2.5_per']].corr('spearman')


    cor_x_cu_p2=data2[['Cu_nm','PM2.5_nm']].corr()
    cor_x_s_p2=data2[['S_nm','PM2.5_nm']].corr()
    cor_x_zn_p2=data2[['Zn_nm','PM2.5_nm']].corr()
    cor_x_ni_p2=data2[['Ni_nm','PM2.5_nm']].corr()
    cor_x_al_p2=data2[['Al_nm','PM2.5_nm']].corr()
    cor_x_ca_p2=data2[['Ca_nm','PM2.5_nm']].corr()

    cor_x_cu_s2=data2[['Cu_nm','PM2.5_nm']].corr('spearman')
    cor_x_s_s2=data2[['S_nm','PM2.5_nm']].corr('spearman')
    cor_x_zn_s2=data2[['Zn_nm','PM2.5_nm']].corr('spearman')
    cor_x_ni_s2=data2[['Ni_nm','PM2.5_nm']].corr('spearman')
    cor_x_al_s2=data2[['Al_nm','PM2.5_nm']].corr('spearman')
    cor_x_ca_s2=data2[['Ca_nm','PM2.5_nm']].corr('spearman')


    ## put the validation study dataset and correlations together in a list and then return
    final=[data2,n,cor_x_cu_p.iloc[0,1],cor_x_s_p.iloc[0,1],cor_x_zn_p.iloc[0,1],cor_x_ni_p.iloc[0,1],cor_x_al_p.iloc[0,1],cor_x_ca_p.iloc[0,1],
          cor_x_cu_s.iloc[0,1],cor_x_s_s.iloc[0,1],cor_x_zn_s.iloc[0,1],cor_x_ni_s.iloc[0,1],cor_x_al_s.iloc[0,1],cor_x_ca_s.iloc[0,1],
          cor_x_cu_p2.iloc[0,1],cor_x_s_p2.iloc[0,1],cor_x_zn_p2.iloc[0,1],cor_x_ni_p2.iloc[0,1],cor_x_al_p2.iloc[0,1],cor_x_ca_p2.iloc[0,1],
          cor_x_cu_s2.iloc[0,1],cor_x_s_s2.iloc[0,1],cor_x_zn_s2.iloc[0,1],cor_x_ni_s2.iloc[0,1],cor_x_al_s2.iloc[0,1],cor_x_ca_s2.iloc[0,1],]

    return final

In [5]:
###########linear regression
#lm
def lms(data):
    data2=data[['Y','PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm','age','race']]
    y=data2['Y']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm','age','race']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

#lm_interactions
def lms_iq(data):
    data2=data[['Y','PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm','age','race']]
    data2['Cu_a']=data2['Cu_nm']*data2['age']
    data2['Zn_a']=data2['Zn_nm']*data2['age']
    data2['Al_a']=data2['Al_nm']*data2['Ca_nm']
    
    data2['S_a']=data2['S_nm']*data2['age']
    data2['Ni_a']=data2['Ni_nm']*data2['age']
    data2['Ca_a']=data2['Ca_nm']*data2['age']
    
    data2['Cu2']=data2['Cu_nm']**2
    data2['S2']=data2['S_nm']**2
    data2['Zn2']=data2['Zn_nm']**2
    data2['Ni2']=data2['Ni_nm']**2
    data2['Al2']=data2['Al_nm']**2
    data2['Ca2']=data2['Ca_nm']**2
    
    y=data2['Y']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm','age','race',
             'Cu_a','Zn_a','Al_a','S_a','Ni_a','Ca_a']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

#lm(+me)
def lm_me(m,v):

    data3=m[['Y','PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm','age','race']]
    data2=v[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm',
              'PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per']]

    y=data2['PM2.5_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x =sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['PM2.5_per']=model.predict(x1)

    y=data2['Cu_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x =sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Cu_per']=model.predict(x1)

    y=data2['S_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['S_per']=model.predict(x1)

    y=data2['Zn_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Zn_per']=model.predict(x1)

    y=data2['Ni_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Ni_per']=model.predict(x1)

    y=data2['Al_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Al_per']=model.predict(x1)

    y=data2['Ca_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Ca_per']=model.predict(x1)

    y=data3['Y']
    x=data3[['PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per','age','race']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()

    return model

#lm_interactions(+me)
def lm_me_iq(m,v):

    data3=m[['Y','PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm','age','race']]
    data2=v[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm',
              'PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per']]

    y=data2['PM2.5_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x =sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['PM2.5_per']=model.predict(x1)

    y=data2['Cu_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x =sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Cu_per']=model.predict(x1)

    y=data2['S_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['S_per']=model.predict(x1)

    y=data2['Zn_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Zn_per']=model.predict(x1)

    y=data2['Ni_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Ni_per']=model.predict(x1)

    y=data2['Al_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Al_per']=model.predict(x1)

    y=data2['Ca_per']
    x=data2[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    x1=sm.add_constant(data3[['PM2.5_nm','Cu_nm','S_nm','Zn_nm','Ni_nm','Al_nm','Ca_nm']])
    data3['Ca_per']=model.predict(x1)

    data3['Cu_a']=data3['Cu_per']*data3['age']
    data3['Zn_a']=data3['Zn_per']*data3['age']
    data3['Al_a']=data3['Al_per']*data3['age']
    
    data3['S_a']=data3['S_per']*data3['age']
    data3['Ni_a']=data3['Ni_per']*data3['age']
    data3['Ca_a']=data3['Ca_per']*data3['age']
    
    data3['Cu2']=data3['Cu_per']**2
    data3['S2']=data3['S_per']**2
    data3['Zn2']=data3['Zn_per']**2
    data3['Ni2']=data3['Ni_per']**2
    data3['Al2']=data3['Al_per']**2
    data3['Ca2']=data3['Ca_per']**2

    y=data3['Y']

    x=data3[['PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per','age','race',
             'Cu_a','Zn_a','Al_a','S_a','Ni_a','Ca_a']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()

    return model


#lm_per(true)
def lms_per(data):
    data2=data[['Y','PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per','age','race']]
    y=data2['Y']
    x=data2[['PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per','age','race']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model


#lm_interactions_per(true)
def lms_iq_per(data):
    data2=data[['Y','PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per','age','race']]
    data2['Cu_a']=data2['Cu_per']*data2['age']
    data2['Zn_a']=data2['Zn_per']*data2['age']
    data2['Al_a']=data2['Al_per']*data2['age']
    
    data2['S_a']=data2['S_per']*data2['age']
    data2['Ni_a']=data2['Ni_per']*data2['age']
    data2['Ca_a']=data2['Ca_per']*data2['age']
    
    data2['Cu2']=data2['Cu_per']**2
    data2['S2']=data2['S_per']**2
    data2['Zn2']=data2['Zn_per']**2
    data2['Ni2']=data2['Ni_per']**2
    data2['Al2']=data2['Al_per']**2
    data2['Ca2']=data2['Ca_per']**2
    
    y=data2['Y']
    x=data2[['PM2.5_per','Cu_per','S_per','Zn_per','Ni_per','Al_per','Ca_per','age','race',
             'Cu_a','Zn_a','Al_a','S_a','Ni_a','Ca_a']]
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

In [6]:
# 3_R-squared=0.4 (scenario 1)
n1=350
n2=1000

cor12=0.3
cor13=0.73
cor14=0.51
cor15=0.07
cor16=-0.15
cor17=0.17
            
cor23=0.18
cor24=0.47
cor25=0.24
cor26=0.13
cor27=0.53

cor34=0.48
cor35=0.09
cor36=-0.16
cor37=0.08

cor45=0.35
cor46=-0.04
cor47=0.54

cor56=-0.13
cor57=0.32

cor67=0.57

mean_x=21.14
mean_cu=0.014
mean_s=0.92

mean_zn=0.08
mean_ni=0.004

mean_al=0.08
mean_ca=0.32


sigma_x=16
sigma_cu=0.05
sigma_s=0.6

sigma_zn=0.34
sigma_ni=0.006

sigma_al=0.12
sigma_ca=0.53


cor2=0.1

mean_ep_x=0
mean_ep_cu=0
mean_ep_s=0
mean_ep_zn=0
mean_ep_ni=0
mean_ep_al=0
mean_ep_ca=0

icc_x=0.7
icc_cu=0.7
icc_s=0.7

icc_zn=0.7
icc_ni=0.7
icc_al=0.7

icc_ca=0.7


age_min=18
age_max=91

#####################adjustment#############
b_cu=26
b_s=50
b_zn=50
b_ni=-29
b_al=-26
b_ca=-70
b_age=3
b_race=3

ba_cu=3
ba_s=-0.5
ba_zn=-1
ba_ni=10
ba_al=2
ba_ca=-2

###############
b_ca12=200
b_s12=-20
b_zn12=-50
b_ni12=1000
b_al12=-50
b_cu12=-395
             
b_cu_s=180
b_zn_ni=100
b_al_ca=100
b_s_zn=-25
b_ni_al=-100
b_ca_cu=-100
b_cu2=600
b_s2=-250
b_zn2=100
b_ni2=-10
b_al2=-150
b_ca2=800
#################
###########################

b_0=1
b_x=0.77
sd_xi=27

In [8]:
# scenario 1

lm1=[]
lm2=[]
lm3=[]
lm4=[]
lm5=[]
lm6=[]
mmm=[]

v=ge_left2(n1,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,
             
             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,
             
             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2)

for i in range(1):
    
    m=ge_main(n2,age_min,age_max,
            mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
            sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,
            
            cor12,cor13,cor14,cor15,cor16,cor17,
            cor23,cor24,cor25,cor26,cor27,
            cor34,cor35,cor36,cor37,
            cor45,cor46,cor47,
            cor56,cor57,
            cor67,
                  
            mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
            icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2,
                  
            b_cu,b_s,b_zn,b_ni,b_al,b_ca,b_age,b_race,
            b_cu_s,b_zn_ni,b_al_ca,b_s_zn,b_ni_al,b_ca_cu,b_cu2,b_s2,b_zn2,b_ni2,b_al2,b_ca2,
            b_0,b_x,sd_xi,b_ca12,b_s12,b_zn12,b_cu12,b_al12,b_ni12,
             ba_cu,ba_s,ba_zn,ba_ni,ba_al,ba_ca)
    
    a1=lms(m[0])
    a2=lms_iq(m[0]) 
    a3=lm_me(m[0],v[0]) 
    a4=lm_me_iq(m[0],v[0])
    a5=lms_per(m[0])
    a6=lms_iq_per(m[0])
    
    lm1.append(a1)
    lm2.append(a2)
    lm3.append(a3)
    lm4.append(a4)
    lm5.append(a5)
    lm6.append(a6)
    mmm.append(m[0])


ll=[lm1,lm2,lm3,lm4,lm5,lm6,mmm]
fw = open('lmv1.pkl','wb')
pickle.dump(ll, fw)
fw.close()

NameError: name 'sm' is not defined

In [1]:
354.19/60 # time

5.9031666666666665

In [8]:
# 1-2
# 3_R-squared=0.4 (scenario 2)
n1=350
n2=1000

cor12=0.3
cor13=0.73
cor14=0.51
cor15=0.07
cor16=-0.15
cor17=0.17
            
cor23=0.18
cor24=0.47
cor25=0.24
cor26=0.13
cor27=0.53

cor34=0.48
cor35=0.09
cor36=-0.16
cor37=0.08

cor45=0.35
cor46=-0.04
cor47=0.54

cor56=-0.13
cor57=0.32

cor67=0.57

mean_x=21.14
mean_cu=0.014
mean_s=0.92

mean_zn=0.08
mean_ni=0.004

mean_al=0.08
mean_ca=0.32


sigma_x=16
sigma_cu=0.05
sigma_s=0.6

sigma_zn=0.34
sigma_ni=0.006

sigma_al=0.12
sigma_ca=0.53


cor2=0.1

mean_ep_x=0
mean_ep_cu=0
mean_ep_s=0
mean_ep_zn=0
mean_ep_ni=0
mean_ep_al=0
mean_ep_ca=0

icc_x=0.7
icc_cu=0.7
icc_s=0.7

icc_zn=0.7
icc_ni=0.7
icc_al=0.7

icc_ca=0.7


age_min=18
age_max=91

#####################adjustment#############
b_cu=5
b_s=10
b_zn=10
b_ni=-7
b_al=-5
b_ca=-10
b_age=3
b_race=4

ba_cu=1
ba_s=-0.1
ba_zn=-0.2
ba_ni=2
ba_al=0.3
ba_ca=-0.2

#################
b_ca12=200
b_s12=-20
b_zn12=-50
b_ni12=1000
b_al12=-50
b_cu12=-395
             
b_cu_s=180
b_zn_ni=100
b_al_ca=100
b_s_zn=-25
b_ni_al=-100
b_ca_cu=-100
b_cu2=600
b_s2=-250
b_zn2=100
b_ni2=-10
b_al2=-150
b_ca2=800
#################
###########################

b_0=1
b_x=0.77
sd_xi=14

In [9]:
# scenario 2
t_s=time()

lm1=[]
lm2=[]
lm3=[]
lm4=[]
lm5=[]
lm6=[]
mmm=[]

v=ge_left2(n1,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,
             
             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,
             
             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2)

for i in range(100):
    
    m=ge_main(n2,age_min,age_max,
            mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
            sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,
            
            cor12,cor13,cor14,cor15,cor16,cor17,
            cor23,cor24,cor25,cor26,cor27,
            cor34,cor35,cor36,cor37,
            cor45,cor46,cor47,
            cor56,cor57,
            cor67,
                  
            mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
            icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2,
                  
            b_cu,b_s,b_zn,b_ni,b_al,b_ca,b_age,b_race,
            b_cu_s,b_zn_ni,b_al_ca,b_s_zn,b_ni_al,b_ca_cu,b_cu2,b_s2,b_zn2,b_ni2,b_al2,b_ca2,
            b_0,b_x,sd_xi,b_ca12,b_s12,b_zn12,b_cu12,b_al12,b_ni12,
             ba_cu,ba_s,ba_zn,ba_ni,ba_al,ba_ca)
    
    a1=lms(m[0])
    a2=lms_iq(m[0]) 
    a3=lm_me(m[0],v[0]) 
    a4=lm_me_iq(m[0],v[0])
    a5=lms_per(m[0])
    a6=lms_iq_per(m[0])
    
    lm1.append(a1)
    lm2.append(a2)
    lm3.append(a3)
    lm4.append(a4)
    lm5.append(a5)
    lm6.append(a6)
    mmm.append(m[0])

t_e=time()

ll=[lm1,lm2,lm3,lm4,lm5,lm6,mmm]
fw = open('lmv12.pkl','wb')
pickle.dump(ll, fw)
fw.close()

print("time in seconds:",t_e-t_s)

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

...

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

time in seconds: 349.11514377593994


In [2]:
349.115/60 # time

5.818583333333334

In [11]:
# 1-3
# 3_R-squared=0.4 (scenario 3)
n1=350
n2=1000

cor12=0.3
cor13=0.73
cor14=0.51
cor15=0.07
cor16=-0.15
cor17=0.17
            
cor23=0.18
cor24=0.47
cor25=0.24
cor26=0.13
cor27=0.53

cor34=0.48
cor35=0.09
cor36=-0.16
cor37=0.08

cor45=0.35
cor46=-0.04
cor47=0.54

cor56=-0.13
cor57=0.32

cor67=0.57

mean_x=21.14
mean_cu=0.014
mean_s=0.92

mean_zn=0.08
mean_ni=0.004

mean_al=0.08
mean_ca=0.32


sigma_x=16
sigma_cu=0.05
sigma_s=0.6

sigma_zn=0.34
sigma_ni=0.006

sigma_al=0.12
sigma_ca=0.53


cor2=0.1

mean_ep_x=0
mean_ep_cu=0
mean_ep_s=0
mean_ep_zn=0
mean_ep_ni=0
mean_ep_al=0
mean_ep_ca=0

icc_x=0.7
icc_cu=0.7
icc_s=0.7

icc_zn=0.7
icc_ni=0.7
icc_al=0.7

icc_ca=0.7


age_min=18
age_max=91

#####################adjustment#############
b_cu=15
b_s=30
b_zn=25
b_ni=-15
b_al=-13
b_ca=-38
b_age=4
b_race=4

ba_cu=3
ba_s=-0.3
ba_zn=-0.7
ba_ni=10
ba_al=0.8
ba_ca=-0.8

###################
b_ca12=200
b_s12=-20
b_zn12=-50
b_ni12=1000
b_al12=-50
b_cu12=-395
             
b_cu_s=180
b_zn_ni=100
b_al_ca=100
b_s_zn=-25
b_ni_al=-100
b_ca_cu=-100
b_cu2=600
b_s2=-250
b_zn2=100
b_ni2=-10
b_al2=-150
b_ca2=800
######################
###########################

b_0=1
b_x=0.77
sd_xi=17

In [12]:
# scenario 3
t_s=time()

lm1=[]
lm2=[]
lm3=[]
lm4=[]
lm5=[]
lm6=[]
mmm=[]

v=ge_left2(n1,mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
             sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,
             
             cor12,cor13,cor14,cor15,cor16,cor17,
             cor23,cor24,cor25,cor26,cor27,
             cor34,cor35,cor36,cor37,
             cor45,cor46,cor47,
             cor56,cor57,
             cor67,
             
             mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
             icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2)

for i in range(100):
    
    m=ge_main(n2,age_min,age_max,
            mean_x,mean_cu,mean_s,mean_zn,mean_ni,mean_al,mean_ca,
            sigma_x,sigma_cu,sigma_s,sigma_zn,sigma_ni,sigma_al,sigma_ca,
            
            cor12,cor13,cor14,cor15,cor16,cor17,
            cor23,cor24,cor25,cor26,cor27,
            cor34,cor35,cor36,cor37,
            cor45,cor46,cor47,
            cor56,cor57,
            cor67,
                  
            mean_ep_x,mean_ep_cu,mean_ep_s,mean_ep_zn,mean_ep_ni,mean_ep_al,mean_ep_ca,
            icc_x,icc_cu,icc_s,icc_zn,icc_ni,icc_al,icc_ca,cor2,
                  
            b_cu,b_s,b_zn,b_ni,b_al,b_ca,b_age,b_race,
            b_cu_s,b_zn_ni,b_al_ca,b_s_zn,b_ni_al,b_ca_cu,b_cu2,b_s2,b_zn2,b_ni2,b_al2,b_ca2,
            b_0,b_x,sd_xi,b_ca12,b_s12,b_zn12,b_cu12,b_al12,b_ni12,
             ba_cu,ba_s,ba_zn,ba_ni,ba_al,ba_ca)
    
    a1=lms(m[0])
    a2=lms_iq(m[0]) 
    a3=lm_me(m[0],v[0]) 
    a4=lm_me_iq(m[0],v[0])
    a5=lms_per(m[0])
    a6=lms_iq_per(m[0])
    
    lm1.append(a1)
    lm2.append(a2)
    lm3.append(a3)
    lm4.append(a4)
    lm5.append(a5)
    lm6.append(a6)
    mmm.append(m[0])

t_e=time()

ll=[lm1,lm2,lm3,lm4,lm5,lm6,mmm]
fw = open('lmv13.pkl','wb')
pickle.dump(ll, fw)
fw.close()

print("time in seconds:",t_e-t_s)

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

...

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

<ipython-input-4-7e9fa18a52d5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Cu_a']=data2['Cu_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Zn_a']=data2['Zn_nm']*data2['age']
<ipython-input-4-7e9fa18a52d5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

time in seconds: 346.93231987953186


In [3]:
346.93/60 # time

5.782166666666667

In [ ]:
    data3['gww']=b_cu*(data3['Cu_per'].apply(lambda X: sin(70*X*pi)))
    +b_s*(data3['S_per'].apply(lambda X: sin(4.2*X*pi)))+b_zn*(data3['Zn_per'].apply(lambda X: sin(9*X*pi)))
    +b_ni*(data3['Ni_per'].apply(lambda X: sin(560*X*pi)))+b_al*(data3['Al_per'].apply(lambda X: sin(30*X*pi)))

    +b_ca*(data3['Ca_per'].apply(lambda X: sin(5.6*X*pi)))+b_race*data3['race']
    +ba_cu*data3['age']*data3['Cu_per']+ba_s*data3['age']*data3['S_per']+ba_zn*data3['age']*data3['Zn_per']
    +ba_ni*data3['age']*data3['Ni_per']+ba_al*data3['age']*data3['Al_per']+ba_ca*data3['age']*data3['Ca_per']

In [9]:
data3=m[0]

In [10]:
data3['gww']-b_cu*(data3['Cu_per'].apply(lambda X: sin(70*X*pi)))

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
      ... 
970    0.0
971    0.0
972    0.0
973    0.0
974    0.0
975    0.0
976    0.0
977    0.0
978    0.0
979    0.0
980    0.0
981    0.0
982    0.0
983    0.0
984    0.0
985    0.0
986    0.0
987    0.0
988    0.0
989    0.0
990    0.0
991    0.0
992    0.0
993    0.0
994    0.0
995    0.0
996    0.0
997    0.0
998    0.0
999    0.0
Length: 1000, dtype: float64

In [18]:
a=1+2
+4
print(a)

3


In [17]:
a=1+2+3
print(a)

6
